Load Julia packages (libraries) needed  for the snippets in chapter 0

In [1]:
using StanModels

┌ Info: Recompiling stale cache file /Users/rob/.julia/compiled/v1.3/StanModels/PqEyX.ji for StanModels [fb740163-aa3c-59c1-9c12-c3f890714cde]
└ @ Base loading.jl:1240


CmdStan uses a tmp directory to store the output of cmdstan

In [2]:
ProjDir = rel_path_s("..", "scripts", "05")
cd(ProjDir)

### snippet 5.1

In [3]:
wd = CSV.read(rel_path("..", "data", "WaffleDivorce.csv"), delim=';')
df = convert(DataFrame, wd);

mean_ma = mean(df[:Marriage])
df[:Marriage_s] = convert(Vector{Float64},
  (df[:Marriage]) .- mean_ma)/std(df[:Marriage]);

mean_mam = mean(df[:MedianAgeMarriage])
df[:MedianAgeMarriage_s] = convert(Vector{Float64},
  (df[:MedianAgeMarriage]) .- mean_mam)/std(df[:MedianAgeMarriage]);

df[1:6, [1, 7, 14, 15]]

rethinking_data = "
    Location Divorce  Marriage.s MedianAgeMarriage.s
1    Alabama    12.7  0.02264406          -0.6062895
2     Alaska    12.5  1.54980162          -0.6866993
3    Arizona    10.8  0.04897436          -0.2042408
4   Arkansas    13.5  1.65512283          -1.4103870
5 California     8.0 -0.26698927           0.5998567
"

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at string:4
└ @ Core string:4
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at string:5
└ @ Core string:5
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at string:5
└ @ Core string:5
┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = top-level scope at string:5
└ @ Core string:5
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at string:8
└ @ Core string:8
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope a

"\n    Location Divorce  Marriage.s MedianAgeMarriage.s\n1    Alabama    12.7  0.02264406          -0.6062895\n2     Alaska    12.5  1.54980162          -0.6866993\n3    Arizona    10.8  0.04897436          -0.2042408\n4   Arkansas    13.5  1.65512283          -1.4103870\n5 California     8.0 -0.26698927           0.5998567\n"

Define the Stan language model

In [4]:
m5_3 = "
data {
  int N;
  vector[N] divorce;
  vector[N] marriage_z;
  vector[N] median_age_z;
}
parameters {
  real a;
  real bA;
  real bM;
  real<lower=0> sigma;
}
model {
  vector[N] mu = a + median_age_z * bA + marriage_z * bM;
  sigma ~ uniform( 0 , 10 );
  bA ~ normal( 0 , 1 );
  bM ~ normal( 0 , 1 );
  a ~ normal( 10 , 10 );
  divorce ~ normal( mu , sigma );
}
";

Define the Stanmodel and set the output format to :mcmcchains.

In [5]:
stanmodel = Stanmodel(name="m5_3",
monitors = ["a", "bA", "bM", "sigma", "Divorce"],
 model=m5_3, output_format=:mcmcchains);
# Input data for cmdstan
m5_3_data = Dict("N" => size(df, 1), "divorce" => df[:Divorce],
    "marriage_z" => df[:Marriage_s], "median_age_z" => df[:MedianAgeMarriage_s]);
# Sample using cmdstan
rc, chn, cnames = stan(stanmodel, m5_3_data, ProjDir, diagnostics=false,
  CmdStanDir=CMDSTAN_HOME);
# Describe the draws
describe(chn)

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at string:5
└ @ Core string:5
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at string:5
└ @ Core string:5
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at string:5
└ @ Core string:5
Inference for Stan model: m5_3_model
4 chains: each with iter=(1000,1000,1000,1000); warmup=(0,0,0,0); thin=(1,1,1,1); 4000 iterations saved.

Warmup took (0.11, 0.10, 0.14, 0.11) seconds, 0.46 seconds total
Sampling took (0.12, 0.12, 0.10, 0.11) seconds, 0.46 seconds total

                 Mean     MCSE  StdDev     5%    50%    95%    N_Eff  N_Eff/s    R_hat
lp__              -46  3.4e-02     1.5    -49    -45    -44  1.9e+03  4.3e+03  1.0e+00
accept_stat__    0.92  1.4e-03   0.097   0.72   0.96    1.0  

2-element Array{ChainDataFrame,1}

Summary Statistics
. Omitted printing of 1 columns
│ Row │ parameters │ mean      │ std      │ naive_se   │ mcse       │ ess     │
│     │ Symbol     │ Float64   │ Float64  │ Float64    │ Float64    │ Any     │
├─────┼────────────┼───────────┼──────────┼────────────┼────────────┼─────────┤
│ 1   │ a          │ 9.6782    │ 0.214287 │ 0.00338817 │ 0.00383758 │ 3571.12 │
│ 2   │ bA         │ -1.12027  │ 0.29262  │ 0.00462672 │ 0.00503276 │ 2906.91 │
│ 3   │ bM         │ -0.125319 │ 0.290469 │ 0.00459272 │ 0.00472358 │ 2977.93 │
│ 4   │ sigma      │ 1.52296   │ 0.166067 │ 0.00262574 │ 0.0031142  │ 3065.54 │

Quantiles

│ Row │ parameters │ 2.5%      │ 25.0%     │ 50.0%    │ 75.0%     │ 97.5%     │
│     │ Symbol     │ Float64   │ Float64   │ Float64  │ Float64   │ Float64   │
├─────┼────────────┼───────────┼───────────┼──────────┼───────────┼───────────┤
│ 1   │ a          │ 9.24794   │ 9.53889   │ 9.68239  │ 9.81749   │ 10.0912   │
│ 2   │ bA         │ -

Rethinking results

In [6]:
rethinking_results = "
       mean   sd  5.5% 94.5% n_eff Rhat
a      9.69 0.22  9.34 10.03  1313    1
bR    -0.12 0.30 -0.60  0.35   932    1
bA    -1.13 0.29 -1.56 -0.67   994    1
sigma  1.53 0.16  1.28  1.80  1121    1
"

"\n       mean   sd  5.5% 94.5% n_eff Rhat\na      9.69 0.22  9.34 10.03  1313    1\nbR    -0.12 0.30 -0.60  0.35   932    1\nbA    -1.13 0.29 -1.56 -0.67   994    1\nsigma  1.53 0.16  1.28  1.80  1121    1\n"

End of `05/5.3s.jl`

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*